# Cours TAL – Labo 7 : Classification de dépêches d’agence avec NLTK

**Objectifs**
L’objectif de ce labo est de réaliser des expériences de classification de documents avec la boîte à
outils NLTK sur le corpus de dépêches Reuters. Le labo est à effectuer en binôme. Le rendu sera un
notebook Jupyter présentant vos choix, votre code, vos résultats et les discussions. Le labo sera jugé
sur la qualité des expériences et sur la discussion des différentes options explorées.


# 1. Récupération des données

**Données :** les dépêches du corpus Reuters, tel qu’il est fourni par NLTK. Veuillez respecter la
division en données d’entraînement et données de test.

In [20]:
import nltk
from nltk.corpus import reuters
from nltk.stem import WordNetLemmatizer

nltk.download('reuters')
nltk.download('stopwords')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

stop_words = set(nltk.corpus.stopwords.words('english'))

documents = [(list(reuters.words(fileid)), category)
             for category in reuters.categories()
             for fileid in reuters.fileids(category)]

documents_no_stop_words = [(list(filter(lambda w: w.lower() not in stop_words, reuters.words(fileid))), category)
             for category in reuters.categories()
             for fileid in reuters.fileids(category)]

#TODO: Ca marche pas
documents_lemmatized = [(list(map(lemmatizer.lemmatize, reuters.words(fileid))), category)
             for category in reuters.categories()
             for fileid in reuters.fileids(category)]

# Modifie la liste des documents pour n'avoir que catégorie
def documents_with_binary_category(category):
    return [(d, category if c == category else 'other') for (d, c) in documents]

[nltk_data] Downloading package reuters to
[nltk_data]     C:\Users\babac\AppData\Roaming\nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\babac\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\babac\AppData\Roaming\nltk_data...


In [18]:



# random.shuffle(documents) # Mélange des documents pour ne pas récupérer les mêmes données de tests à chaque exécution

# Récupère la fréquence des mots
# all_words = nltk.FreqDist(w.lower() for w in reuters.words()) #TODO: HYPERPARAMETRE lowercase
# On sélectionne les 2000 mots les plus fréquents comme features pour le classifieur
# word_features = list(all_words)[:2000]

# Retourne un dictionnaire indiquant si une feature est présent dans le document
# def document_contains_features(document):
#     doc_words = set(document)
#     features = {}
#     for word in word_features:
#         features['contains({})'.format(word)] = (word in doc_words)
#     return features
# 
# featureset = document_contains_features(documents[0][0])
# print(featureset)
 
# featuresets = [(document_contains_features(d), c) for (d,c) in documents]
# print(featuresets)
# train_set, test_set = featuresets[1000:], featuresets[:1000]
# 
# classifier = nltk.NaiveBayesClassifier.train(train_set)
# print(nltk.classify.accuracy(classifier, test_set))
# classifier.show_most_informative_features(5)


['acq', 'alum', 'barley', 'bop', 'carcass', 'castor-oil', 'cocoa', 'coconut', 'coconut-oil', 'coffee', 'copper', 'copra-cake', 'corn', 'cotton', 'cotton-oil', 'cpi', 'cpu', 'crude', 'dfl', 'dlr', 'dmk', 'earn', 'fuel', 'gas', 'gnp', 'gold', 'grain', 'groundnut', 'groundnut-oil', 'heat', 'hog', 'housing', 'income', 'instal-debt', 'interest', 'ipi', 'iron-steel', 'jet', 'jobs', 'l-cattle', 'lead', 'lei', 'lin-oil', 'livestock', 'lumber', 'meal-feed', 'money-fx', 'money-supply', 'naphtha', 'nat-gas', 'nickel', 'nkr', 'nzdlr', 'oat', 'oilseed', 'orange', 'palladium', 'palm-oil', 'palmkernel', 'pet-chem', 'platinum', 'potato', 'propane', 'rand', 'rape-oil', 'rapeseed', 'reserves', 'retail', 'rice', 'rubber', 'rye', 'ship', 'silver', 'sorghum', 'soy-meal', 'soy-oil', 'soybean', 'strategic-metal', 'sugar', 'sun-meal', 'sun-oil', 'sunseed', 'tea', 'tin', 'trade', 'veg-oil', 'wheat', 'wpi', 'yen', 'zinc']


## 2. Classifieurs binaires

### 2.1 Classifieur binaire pour la catégorie 'money-fx'

#### 2.1.1 Définition des hyperparamètres